#### Vicky van der Wagt

# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. 
In some of these cells you will have questions to answer. 
The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. 
*Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [10]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [11]:
# Use this cell for any import statements you add

#for removing folders and contents
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [12]:
artists = {'kacey musgraves':"https://www.azlyrics.com/k/kaceymusgraves.html",
           'sabrina carpenter':"https://www.azlyrics.com/s/sabrinacarpenter.html"} 
# we'll use this dictionary to hold both the artist name and link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: **It is allowed, because it specified that all users (*) are not allowed to go to pages in the /lyricsdb/ and /song/ directory, but are allowed to go to all other pages/directories that are not marked as disallow. It does not allow user-agent: 008 to visit any of the pages, so this is likely the website owner blocking specific users or robots.**


In [13]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    '''this code does stuff'''
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # Extract the links to lyrics pages from this page
    # and store them in lyrics_pages
    soup = BeautifulSoup(r.content, 'html.parser')
    artist_links = [
        a['href'] 
        for a in soup.find_all('a', href=True) 
        if a['href'].startswith('/lyrics/')
    ]
    
    # Add the extracted links to the lyrics_pages dictionary
    lyrics_pages[artist] = artist_links

Let's make sure we have enough lyrics pages to scrape. 

In [14]:
#if asserted, no error will return
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [15]:
# Let's see how long it's going to take to pull these lyrics 
# If we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take "
          f"{round(len(links)*10/3600,2)} hours.")

For kacey musgraves we have 122.
The full pull will take for this artist will take 0.34 hours.
For sabrina carpenter we have 98.
The full pull will take for this artist will take 0.27 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page. 
    * Then write the lyrics to a text file with that name. 


In [16]:
def generate_filename_from_link(link) :
    #Generates a filename based on the lyric page title (song title)
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [17]:
# Make the lyrics folder here. If you'd like to practice your programming, add 
#functionality that checks to see if the folder exists. If it does, then use 
#shutil.rmtree to remove it and create a new one.

#if lyrics folder exists:
if os.path.isdir("lyrics") : 
    #remove the folder and its contents
    shutil.rmtree("lyrics/")

#make the lyrics folder
os.mkdir("lyrics")


In [18]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

#can replace with a limited number if desired
total_pages = 25

for artist, lyric_pages in lyrics_pages.items() :
    # 1. Build a subfolder for the artist
    # if folder already exists...
    if os.path.isdir(os.path.join("lyrics", artist)) : 
        #remove the folder and its contents
        shutil.rmtree(os.path.join("lyrics", artist))
    # make artist subfolder inside lyrics folder
    os.mkdir(os.path.join("lyrics", artist))
    
    #sets a limit for number of lyric pages to analyze
    #can comment out if no limit desired
    #limit is set above the 'for' loop
    lyric_pages = lyric_pages[:total_pages]
    
    # Use this space to carry out the following steps: 
    
    # 2. Iterate over the lyrics pages

    for lyric_page_url in lyric_pages:
        
        #specify the full url
        full_lyric_page_url = url_stub + lyric_page_url
    
    # 3. Request the lyrics page. 
        # Don't forget to add a line like 
            #`time.sleep(5 + 10*random.random())`to sleep after making the request
        
        r = requests.get(full_lyric_page_url)
        time.sleep(5 + 10 * random.random())
        
        # 4. Extract the title and lyrics from the page.

        soup = BeautifulSoup(r.content, 'html.parser')
            #extract song title (second bold element). remove quotes
        title = soup.find_all("b")[1].get_text().replace('"','')
            
            #extract text from lyrics - 23rd div element
        lyrics = soup.find_all("div")[22].get_text()


         # 5. Write out the title, two returns ('\n'), and the lyrics. 
         # Use `generate_filename_from_link to generate the filename. 
        
        filename = generate_filename_from_link(lyric_page_url)
        with open(os.path.join("lyrics", artist, filename), 'w') as f:
                f.write(title + '\n\n' + lyrics)
    
    
    # Remember to pull at least 20 songs per artist. 
    # It may be fun to pull all songs for the artist
    

In [19]:
print(f"Total run time was "
      f"{round((time.time() - start)/3600,2)} hours.")

Total run time was 0.15 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  
After you have finished the above sections , run all the cells in this notebook. 
Print this to PDF and submit it, per the instructions.

In [20]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html

#returns sequences of alphanumeric characters, converts to lowercase
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the 
directory:
`/lyrics/[Artist Name]/[filename from URL]`. 
This code summarizes the information at a high level to help the instructor evaluate your work. 

In [21]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if 
                  os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f 
                    or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, "
          f"{len(set(artist_words))} are unique.")


For kacey musgraves we have 25 files.
For kacey musgraves we have roughly 6219 words, 985 are unique.
For sabrina carpenter we have 25 files.
For sabrina carpenter we have roughly 9358 words, 873 are unique.
